In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG Engine Management
@forusone (shins777@gmail.com).

https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api#python_1



### Install Vertex AI SDK for Python

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    langchain-google-community \
    google-cloud-discoveryengine \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 k

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash-002"

### Package import and initialize model

In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Tool,
)

# from vertexai.preview.generative_models import (
#     grounding)

# model initialization
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
model = GenerativeModel(MODEL)

## RAG Engine management

### Corpus managment helper functions

In [5]:
# Preview
from vertexai.preview import (
    rag,
    reasoning_engines)

#---------------------------------------------
def get_list_corpora():
  """
  Get corpora list.
  """
  corpora = rag.list_corpora()

  for corpus in corpora:
    print(f"[{corpus.create_time}][{corpus.display_name}][{corpus.name}]")

  return corpora

#---------------------------------------------

def get_corpus(corpus_name):
  """
  Get corpus.
  """
  corpus = rag.get_corpus(name=corpus_name)
  print(corpus)
  return corpus

#---------------------------------------------

def del_corpus(corpus_name):
  """
  Delete corpus.
  """

  rag.delete_corpus(name=corpus_name)
  print(f"Corpus {corpus_name} deleted. takes a little time to check the corpus is deleted.")


#### Get list of corpora that are registered

In [70]:
corpora = get_list_corpora()

#### Get a corpus

In [ ]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/6878122530901590016"
get_corpus(corpus_name)

RagCorpus(name='projects/ai-hangsik/locations/us-central1/ragCorpora/576460752303423488', display_name='Daou_manual', description='', embedding_model_config=EmbeddingModelConfig(publisher_model='projects/ai-hangsik/locations/us-central1/publishers/google/models/text-multilingual-embedding-002', endpoint=None, model=None, model_version_id=None), vector_db=RagManagedDb())


RagCorpus(name='projects/ai-hangsik/locations/us-central1/ragCorpora/576460752303423488', display_name='Daou_manual', description='', embedding_model_config=EmbeddingModelConfig(publisher_model='projects/ai-hangsik/locations/us-central1/publishers/google/models/text-multilingual-embedding-002', endpoint=None, model=None, model_version_id=None), vector_db=RagManagedDb())

#### delete a corpus

In [67]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/5256826665048211456"
del_corpus(corpus_name)

Successfully deleted the RagCorpus.
Corpus projects/ai-hangsik/locations/us-central1/ragCorpora/5256826665048211456 deleted. takes a little time to check the corpus is deleted.


### File management helper functions

In [ ]:
# Preview
from vertexai.preview import (
    rag,
    reasoning_engines)

#---------------------------------------------
def upload_file(corpus_name,
                path,
                display_name,
                description
                ):
  """
  Upload file
  """

  rag_file = rag.upload_file(
    corpus_name=corpus_name,
    path=path,
    display_name=display_name,
    description=description,
  )

  print(rag_file)

  return rag_file

#---------------------------------------------

def import_files(corpus_name,
                 paths,
                 chunk_size,
                 chunk_overlap
                 ):
  """
  Get corpus.

  # corpus_name = "projects/{PROJECT_ID}/locations/us-central1/ragCorpora/{rag_corpus_id}"
  # paths = ["https://drive.google.com/file/123", "gs://my_bucket/my_files_dir"]  # Supports Google Cloud Storage and Google Drive Links

  """

  response = rag.import_files(
      corpus_name=corpus_name,
      paths=paths,
      chunk_size=chunk_size,  # Optional
      chunk_overlap=chunk_overlap,  # Optional
      max_embedding_requests_per_min=900,  # Optional
  )

  print(f"Imported {response.imported_rag_files_count} files.")

  return response

#---------------------------------------------

def get_rag_files(corpus_name):
  """
  Get files
  """

  files = rag.list_files(corpus_name=corpus_name)
  for file in files:
      print(file.display_name)
      print(file.name)

  return files

#---------------------------------------------

def get_rag_file(file_name):
  """
  Get a file.
  """
  rag_file = rag.get_file(name=file_name)
  print(rag_file)
  return rag_file

#---------------------------------------------

def del_lag_file(file_name):
  """
  Delete file.
  """

  rag.delete_file(name=file_name)
  print(f"File {file_name} deleted. takes a little time to check the file is deleted.")


#### upload rag file from local stroage

In [ ]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/4611686018427387904"
path = "./daouoffice_member.pdf"
display_name = "test_corpus dsp"
description = "test_corpus daou desc"

rag_file = upload_file(corpus_name,
                path,
                display_name,
                description
                )
print(rag_file)

RagFile(name='projects/ai-hangsik/locations/us-central1/ragCorpora/4611686018427387904/ragFiles/5309375072799233310', display_name='test_corpus dsp', description='test_corpus daou desc')
RagFile(name='projects/ai-hangsik/locations/us-central1/ragCorpora/4611686018427387904/ragFiles/5309375072799233310', display_name='test_corpus dsp', description='test_corpus daou desc')


#### import rag files from Drive or GCS

In [ ]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/4611686018427387904"
paths = ["gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf",
        "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf",
        ]

rag_files = import_files(corpus_name,
                paths,
                chunk_size=512,
                chunk_overlap=100

                )

print(rag_files)

Imported 2 files.
imported_rag_files_count: 2



#### get rag file list

In [ ]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/576460752303423488"
get_rag_files(corpus_name)

DaouOffice 클라우드 관리자 가이드.pdf
projects/721521243942/locations/us-central1/ragCorpora/576460752303423488/ragFiles/5306831324808626471


ListRagFilesPager<rag_files {
  gcs_source {
    uris: "gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf"
  }
  name: "projects/721521243942/locations/us-central1/ragCorpora/576460752303423488/ragFiles/5306831324808626471"
  display_name: "DaouOffice 클라우드 관리자 가이드.pdf"
  create_time {
    seconds: 1732135225
    nanos: 91839000
  }
  update_time {
    seconds: 1732135225
    nanos: 91839000
  }
  file_status {
    state: ACTIVE
  }
}
>

#### get rag file

In [ ]:
file_name = "projects/721521243942/locations/us-central1/ragCorpora/4611686018427387904/ragFiles/5309369494330206088"
get_rag_file(file_name)

#### delete a rag file

In [ ]:
file_name = "projects/721521243942/locations/us-central1/ragCorpora/4611686018427387904/ragFiles/5309373647786126728"

del_lag_file(file_name)

Successfully deleted the RagFile.
File projects/721521243942/locations/us-central1/ragCorpora/4611686018427387904/ragFiles/5309373647786126728 deleted. takes a little time to check the file is deleted.


### Retrival query

In [ ]:
def retrieve(corpus_name, query):

  response = rag.retrieval_query(
      rag_resources=[
          rag.RagResource(
              rag_corpus=corpus_name,
              # Optional: supply IDs from `rag.list_files()`.
              # rag_file_ids=["rag-file-1", "rag-file-2", ...],
          )
      ],
      text=query,
      similarity_top_k=10,  # Optional
      vector_distance_threshold=0.5,  # Optional
  )
  print(response)

def generate_answer(corpus_name, query):

  rag_retrieval_tool = Tool.from_retrieval(
      retrieval=rag.Retrieval(
          source=rag.VertexRagStore(
              rag_resources=[
                  rag.RagResource(
                      rag_corpus=corpus_name,
                      # Optional: supply IDs from `rag.list_files()`.
                      # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                  )
              ],
              similarity_top_k=3,  # Optional
              vector_distance_threshold=0.5,  # Optional
          ),
      )
  )

  rag_model = GenerativeModel(
      model_name="gemini-1.5-flash-002", tools=[rag_retrieval_tool]
  )
  response = rag_model.generate_content(query)
  print(response.text)


In [ ]:
corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/576460752303423488"
query = "대쉬보드란 무엇인가요?"

retrieve(corpus_name, query)


## Integrate with Reasoning engine

### Helper methods

In [ ]:
# Initialize session history
store = {}

def get_session_history(session_id: str):
  """
    Get session history
  """
  from langchain.memory import ChatMessageHistory


  if session_id not in store:
      store[session_id] = ChatMessageHistory()
  return store[session_id]

#---------------------------------------------------------------
def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']

#---------------------------------------------------------------

def create_agent(rag_corpus_name):
  from vertexai.preview import reasoning_engines

  from googleapiclient import discovery
  from langchain.agents.format_scratchpad.tools import format_to_tool_messages
  from langchain_core import prompts

  # Define prompt template
  prompt = {
      "history": lambda x: x["history"],
      "input": lambda x: x["input"],
      "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
  } | prompts.ChatPromptTemplate.from_messages(
      [
          prompts.MessagesPlaceholder(variable_name="history"),
          ("user", "{input}"),
          prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
      ]
  )

  rag_retrieval_tool = Tool.from_retrieval(
      retrieval=rag.Retrieval(
          source=rag.VertexRagStore(
              rag_resources=[
                  rag.RagResource(
                      rag_corpus=rag_corpus_name,  # Currently only 1 corpus is allowed.
                      # Optional: supply IDs from `rag.list_files()`.
                      # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                  )
              ],
              similarity_top_k=10,  # Optional
              vector_distance_threshold=0.5,  # Optional
          ),
      )
  )

  agent = reasoning_engines.LangchainAgent(
      prompt=prompt,
      model=MODEL,
      chat_history=get_session_history,
      model_kwargs={"temperature": 0},
      tools=[rag_retrieval_tool],
      agent_executor_kwargs={"return_intermediate_steps": True},
  )

  return agent


In [ ]:
from time import perf_counter

session_id = "sess_101"
rag_corpus_name = "projects/ai-hangsik/locations/us-central1/ragCorpora/576460752303423488"

agent = create_agent(rag_corpus_name)

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

사용자: 대쉬보드에 대해서 설명해주세요.


4.678311007999582

AI Agent : 제공된 문서에 따르면, 대시보드는 서비스 페이지에 보이는 가젯과 위젯 등을 관리하는 영역입니다.  대시보드 설정에서는 전사 대시보드 운영자를 지정하고 가젯의 공개 여부를 설정할 수 있으며,  대시보드 운영자는 전사 대시보드를 편집할 수 있는 권한을 가지고 웹 서비스에서 바로 편집 가능합니다.  대시보드에 대한 자세한 설명이나 가젯 설정 방법은 사용자 가이드를 참고해야 합니다.


------------------------------------ 
사용자: 종료
[]


In [ ]:
display_name = "Daou_manual"

paths = ["gs://daou_office_manual/manual_org/DaouOffice 클라우드 관리자 가이드.pdf",
        "gs://daou_office_manual/manual_org/DaouOffice 클라우드 서비스 가이드 3.5.21.pdf",
        ]

rag_corpus, rag = create_rag_corpus(display_name, paths)

print(rag_corpus)
print(rag)


In [ ]:

query = "대쉬보드란 무엇인가요?"

responses = retrieve_manual(rag_corpus, rag, query)

for context in responses.contexts.contexts:
    # print(context)

    print(context.text)
    print(context.distance)

    print('=====')


1. 사이트 어드민의 [메뉴관리>홈 > 대시보드]을 클릭합니다.
2. 운영자 추가를 클릭합니다.
3. 조직도에서 운영자로 지정할 사용자를 모두 선택합니다.
4. 설정이 완료되면 저장을 클릭합니다.
[기본관리>메뉴 운영 권한 관리]에서도 대시보드 운영자를 설정할 수 있습니다.116 | 메뉴 관리
조직도 기능을 사용하지 않는다면, 사용자를 검색하여 추가할 수 있습니다.
대시보드에 대한 좀 더 자세한 설명이나 대시보드에 가젯을 설정하는 방법은 사용자 가이드를 참고하시기
바랍니다.
가젯 공개 설정
그림 6-1 가젯 공개 설정
사용자가 개인 대시보드에서 사용할 수 있는 가젯을 설정합니다. 공개한 가젯만 개인 대시보드에서 사용할 수 있습니
다.전자결재 | 117
1. 사이트 어드민의 [메뉴관리>홈 > 대시보드]을 클릭합니다.
2. 가젯 목록에서 공개, 비공개를 선택합니다.
3. 설정이 완료되면 저장을 클릭합니다.
6.2 전자결재
전자결재란 온라인에서 결재 문서를 작성하고, 결재를 할 수 있는 기능입니다.
0.25556055230334873
=====
결재선은 반드시 입력해야 하는 항목입니다.
■ 텍스트 - 문자를 한 줄로 입력할 수 있는 창입니다.
■ 멀티텍스트 - 문자를 여러 줄로 입력할 수 있는 창입니다.
■ 편집기 - 문자를 입력하거나 서식을 설정할 수 있는 도구입니다. 편집기를 넣지 않으면, 기안자가 글
자를 입력할 수 없습니다. 문자 입력이 필요한 경우에는 반드시 편집기를 넣어야 합니다.
■ 숫자 - 숫자를 입력할 수 있는 창입니다.
■ 통화 - 통화를 입력할 수 있는 창입니다.
■ 단일선택 - 여러 개의 항목 중에 하나를 선택할 수 있는 버튼입니다. 단일 선택을 클릭하면 라디오 버
튼이 삽입됩니다. 컴포넌트를 선택하고 오른쪽에서 A와 B 대신 선택 항목을 입력합니다.
■ 드롭박스 - 여러 개의 항목에서 하나를 선택할 수 있는 버튼입니다.
■ 복수 선택(Check) - 여러 개의 항목에서 하나 이상을 선택할 수 있는 버튼입니다. 복수 선택을 클릭하
면 체크박스가